In [1]:
#import libraries
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [26]:
#Download data
!wget -q -O 'toronto_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


In [93]:
#Read data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto = pd.read_html(url)
toronto_neigh=toronto[0]
toronto_neigh.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [94]:
#Drop not assigned Boroughs
toronto_neigh_NA = toronto_neigh[ toronto_neigh['Borough'] == 'Not assigned' ]
toronto_neigh = toronto_neigh.drop(toronto_neigh_NA.index, axis=0)
#reset index
toronto_neigh.reset_index(drop=True,inplace=True)
toronto_neigh.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [97]:
#group by same Postcode and join Neighbourhood separated with commas
toronto_neigh=toronto_neigh.groupby(['Postcode','Borough'], as_index = False).agg(lambda x: ','.join(set(x)))
toronto_neigh.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [98]:
toronto_neigh['Neighbourhood'].replace('Not assigned',toronto_neigh['Borough'],inplace=True)
toronto_neigh.iloc[85]

Postcode                  M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 85, dtype: object

In [100]:
toronto_neigh.shape

(103, 3)

In [105]:
#Read geographical coordinates file given
url2='http://cocl.us/Geospatial_data'
df_long_lat=pd.read_csv(url2)
df_long_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [115]:
toronto_neigh_complete = pd.concat([toronto_neigh, df_long_lat], axis=1)
toronto_neigh_complete =toronto_neigh_complete[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
toronto_neigh_complete.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
